In [ ]:

import pandas as pd
import numpy as np
import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from unidecode import unidecode
## ler arquivo json em pandas
df = pd.read_json('/home/mfelipemota/projects/pantanaldev/data/label-studio/data/export/project-1-at-2023-05-07-03-57-20fa509e.json')

df = df.drop(['id'], axis=1)
## expandir coluna annotations
df = pd.concat([df.drop(['annotations'], axis=1), df['annotations'].apply(pd.Series)], axis=1)
## expandir coluna 0 e renomear para annotations
df = pd.concat([df.drop([0], axis=1), df[0].apply(pd.Series)], axis=1)
## expandir coluna result e renomear para result
df = pd.concat([df.drop(['result'], axis=1), df['result'].apply(pd.Series)], axis=1)
df = df.drop(['id'], axis=1)

## expandir coluna 0 e renomear para result
df = pd.concat([df.drop([0], axis=1), df[0].apply(pd.Series)], axis=1)
## expandir coluna value e renomear para value
df = pd.concat([df.drop(['value'], axis=1), df['value'].apply(pd.Series)], axis=1)
## dropar choices nulos
df = df.dropna(subset=['choices'])
## obter choices 
df['choices'] = df['choices'].apply(lambda x: x[0])

## expandir coluna data
df = pd.concat([df.drop(['data'], axis=1), df['data'].apply(pd.Series)], axis=1)

df_noticia_original = df.copy()


padrao_data_cepea = r"Cepea, \d{2}/\d{2}/\d{4} - "
df['noticia'] = df['noticia'].apply(lambda x: re.sub(padrao_data_cepea, '', x))

## remover a palavra 'cepea' das noticias
padrao_cepea = r"Cepea"
df['noticia'] = df['noticia'].apply(lambda x: re.sub(padrao_cepea, '', x, flags=re.IGNORECASE))

## remover numeros das noticias
padrao_numeros = r'[0-9]+'
df['noticia'] = df['noticia'].apply(lambda x: re.sub(padrao_numeros, '', x))

## noticia que contem a palavra 'soja'
df = df[df['titulo'].str.contains('soja', flags=re.IGNORECASE)]

## remover noticias com choice 'desclassificar'
df = df[df['choices'] != 'Desclassificar']
df.count()

In [ ]:
# Selecionar apenas as colunas necessárias
columns_to_select = ['id', 'data', 'noticia', 'titulo', 'choices', 'unique_id']

df = df[columns_to_select]
df.dropna(subset=['noticia'])

# Pré-processamento dos dados
stop_words = set(stopwords.words('portuguese'))

def preprocess_text(text):
    # remover acentuação
    text = unidecode(text)
    # Remover pontuações
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenização
    words = word_tokenize(text.lower())
    # Remover stopwords
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

df['noticia'] = df['noticia'].apply(preprocess_text)
df_treino = df[:208]
df_validacao = df[208:]


In [ ]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer  # Or BertTokenizer
from transformers import AutoModelForPreTraining  # Or BertForPreTraining for loading pretraining heads
from transformers import AutoModel  # or BertModel, for BERT without pretraining heads
import numpy as np

from typing import List, Optional, Tuple, Union

# Carregar o modelo pré-treinado BERTimbau
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained("bert-base-multilingual-cased")

modelo_soja = model

modelo_soja.config

/home/mfelipemota/projects/pantanaldev/data/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a mod

BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.28.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 119547
}

In [ ]:
class LIABertClassifier(nn.Module):
    def __init__(self,model,num_labels):
        super(LIABertClassifier,self).__init__()
        self.bert = model
        self.config = model.config
        self.num_labels = num_labels
        self.cls = nn.Linear(self.config.hidden_size,200)
        self.dropout = nn.Dropout(p=0.5)
        self.cls2 = nn.Linear(200,num_labels)

    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        token_type_ids: Optional[torch.Tensor] = None,
        ) ->Tuple[torch.Tensor]:

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        sequence_output = outputs[0][:,0,:]
        prediction = self.cls(sequence_output)
        prediction = self.dropout(prediction)
        prediction = self.cls2(prediction)
        return prediction
        

In [ ]:
torch.cuda.empty_cache()

from torch.utils.data import DataLoader
from transformers import AdamW, get_scheduler
import sklearn.model_selection as model_selection
from torch.optim.lr_scheduler import OneCycleLR

# Map string labels to integer labels
label_map = {'Negativa': 0, 'Positiva': 1, 'Neutra': 2}
ytrain_global = np.array([label_map[label] for label in df_treino['choices']])
xtrain_global = np.array(df_treino['noticia'])



xtrain, xval, ytrain, yval = model_selection.train_test_split(xtrain_global, ytrain_global, test_size=0.30, random_state=42,shuffle=True)

train_encodings = tokenizer(xtrain.tolist(), truncation=True, padding=True, max_length=512, return_tensors='pt')
val_encodings = tokenizer(xval.tolist(), truncation=True, padding=True,max_length=512, return_tensors='pt')

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        label = torch.tensor(self.labels[idx])
        return (item,label)

    def __len__(self):
        return len(self.labels)

train_dataset = MyDataset(train_encodings, ytrain)
val_dataset = MyDataset(val_encodings, yval)

device = torch.device('cpu')

model = LIABertClassifier(modelo_soja, 3)

model.to(device)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

batch = {k: v.to(device) for k, v in x.items()}
model.to(device)

optim = AdamW(model.parameters(), lr=0.000050)

loss_fct = nn.CrossEntropyLoss()

num_epochs = 20

from transformers import get_scheduler

scheduler = get_scheduler(
    "linear",
    optim,
    num_warmup_steps=0,
    num_training_steps=num_epochs*len(dl_train)
)

# scheduler = OneCycleLR(optim, max_lr=max_lr, total_steps=total_steps,
#                        div_factor=div_factor, pct_start=pct_start)

for epoch in range(num_epochs):
    lepochs = []
    for batch,y in dl_train:
        model.train()
        batch = {k: v.to(device) for k, v in batch.items()}
        y = y.to(device)
        outputs = model(**batch)
        loss = loss_fct(outputs,y)
        lepochs.append(loss.cpu().item())
        loss.backward()
        optim.step()
        scheduler.step()
        optim.zero_grad()
        current_lr = optim.param_groups[0]['lr']

        print(f"Epoch {epoch}, Loss={loss:.4f}, LR={current_lr:.6f}")

model.eval()
ytrue = []
ypred = []
for batch,y in dl_eval:
    batch = {k: v.to(device) for k, v in batch.items()}
    y = y.to(device)
    with torch.no_grad():
        outputs = model(**batch)
        ytrue.extend(y.cpu().numpy().tolist())
        ypred.extend(outputs.argmax(axis=1).cpu().numpy().tolist())

for epoch in range(5):
    loss, acc = train_epoch(model, train_loader, optim)
    print(f'Epoch {epoch} train loss {loss} train accuracy {acc}')
    loss, acc = val_epoch(model, val_loader)
    print(f'Epoch {epoch} val loss {loss} val accuracy {acc}')


print(classification_report(y_true, y_pred))


(', // – Os preços da soja estão em alta no Brasil. O Indicador ESALQ/BM&FBovespa da soja Paranaguá (PR) subiu ,% entre  e  de março, a R$ ,/sc de  kg nessa sexta-feira,  – na quarta-feira, , o Indicador atingiu R$ ,/sc, o maior valor nominal da série histórica do , iniciada em março/. Quanto ao Indicador /ESALQ Paraná, foi de R$ ,/sc de  kg na sexta-feira, , ,% superior ao dia . Segundo pesquisadores do , as elevações no Brasil estão atreladas à combinação de firmes demandas externa e doméstica, da alta nos preços futuros e do dólar elevado. Além disso, para combater o avanço do coronavírus, o governo argentino limitou o movimento nos portos do país, cenário que pode favorecer as vendas brasileiras de soja e derivados. Fonte:  – ', 1)
(', // -\xa0Os preços internos da soja seguiram em alta nos últimos dias, mesmo com o avanço da colheita no Brasil. A sustentação veio do mercado internacional. Além do surpreendente aumento da demanda chinesa pela soja, novas estimativas apontam menor p

2023-05-07 01:50:41.718216: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-07 01:50:42.149154: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-07 01:50:43.319144: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/mfelipemota/projects/pantanaldev/data/.venv/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.o

IndexError: Target 2 is out of bounds.